In [ ]:
import os
import re
import pathlib
import json
import pandas as pd
import psycopg2
import csv
from bs4 import BeautifulSoup

In [ ]:
def generate_Fitbit_json_file_lists():
    '''
        takes in directory containing json files for heart rate
        returns a list of paths to each individual file, as strings,
        for easy consumption by next function
    '''
    path = str(pathlib.Path().absolute())
    heart_jsons = []
    calorie_jsons = []
    location_kmls = []

    for loc, dirs, files in os.walk('./json_heart_data'):
        print('\nvisiting location:', loc, '\n')
        # pass the current iteration's object pair (ignoring dirs)
        # to our method dedicated to looking at a list of files 
        # in any arbitrary file location
        print('FILES:', files, '\n')
        regexp = re.compile('(\.json)$')
        for f in files:
            # rebuld the file name from the given directory name
            # and the OS-specific file path separator we can get with os.sep
            # and the filename we get from our iterator
            fn = 'json_heart_data' + str(os.sep) + f
            if os.path.isfile(fn):
                match = re.search(regexp, fn)
                if match:
                    if match.group(0) == '.json':
                        print('file name:', fn, ' size:', os.path.getsize(fn),'B ', end=' ')
                        print('extension:', match.group(0))
                        heart_jsons.append(path + '/' + fn)

    for loc, dirs, files in os.walk('./json_calorie_data'):
        print('\nvisiting location:', loc, '\n')
        print('FILES:', files, '\n')
        regexp = re.compile('(\.json)$')
        for f in files:
            fn = 'json_calorie_data' + str(os.sep) + f
            if os.path.isfile(fn):
                match = re.search(regexp, fn)
                if match:
                    if match.group(0) == '.json':
                        print('file name:', fn, ' size:', os.path.getsize(fn),'B ', end=' ')
                        print('extension:', match.group(0))
                        calorie_jsons.append(path + '/' + fn)

    for loc, dirs, files in os.walk('./kml_google_data'):
        print('\nvisiting location:', loc, '\n')
        print('FILES:', files, '\n')
        regexp = re.compile('(\.kml)$')
        for f in files:
            fn = 'kml_google_data' + str(os.sep) + f
            if os.path.isfile(fn):
                match = re.search(regexp, fn)
                if match:
                    if match.group(0) == '.kml':
                        print('file name:', fn, ' size:', os.path.getsize(fn),'B ', end=' ')
                        print('extension:', match.group(0))
                        location_kmls.append(path + '/' + fn)

    return heart_jsons, calorie_jsons, location_kmls

In [ ]:
def json_heart_convert_csv(heart_json_path):
    
    heart = pd.read_json(heart_json_path)
    # Empty lists for heartbeats/min and Fitbit 'Confidence' value
    bpm_list = []
    conf_list = []

    # Populates empty lists with respective Fitbit values
    for i in heart['value']:
        bpm_list.append(i['bpm'])
        conf_list.append(i['confidence'])

    # Creates new pandas dataframes from populated lists
    bpm_df = pd.DataFrame(bpm_list)
    conf_df = pd.DataFrame(conf_list)

    # Assigns dataframes as new series/columns in original dataframe
    heart['BPM'] = bpm_df
    heart['Confidence'] = conf_df
    
    # Exports dataframes as CSV
    heart_csv = heart[['dateTime','BPM','Confidence']].to_csv('working_heart.csv')
    return heart_csv

In [ ]:
def json_calorie_convert_csv(calorie_json_path):

    # Reads JSON file and renames column for Calories burned per minute
    calor = pd.read_json(calorie_json_path)
    calor.rename(columns={'value':'Calories'}, inplace=True)

    # Exports dataframes as CSV
    calorie_csv = calor.to_csv('working_calorie.csv')

    return calorie_csv

In [ ]:
def csv_format_google_kml(location_kml_path):
    
    name_list = []
    address_list = []
    description_list = []
    time_begin_list = []
    time_end_list = []
    coordinates_list = []
    longitude_list = []
    latitude_list = []
    linestring_coord_list = []

    with open(location_kml_path, 'r') as f: # Raw KML file is read using BeautifulSoup...
            kml = BeautifulSoup(f, 'xml')
            for name in kml.find_all('name')[1:]: # Relevant headers are isolated from KML...
                name_list.append(re.sub('<.*?>', '', str(name))) # Headers are removed...
            for address in kml.find_all('address'):
                address_list.append(re.sub('<.*?>', '', str(address))) # Data strings appended to corresponding lists
            for description in kml.find_all('description')[1:]: # 'Name','Description' headers for whole document excluded from parsing
                description_list.append((re.sub('<.*?>', '', str(description)))[1:]) # Removing extra space at beginning of string
            for time_begin in kml.find_all('begin'):
                time_begin_list.append(re.sub('<.*?>', '', str(time_begin)))
            for time_end in kml.find_all('end'):
                time_end_list.append(re.sub('<.*?>', '', str(time_end)))
            for coordinates in kml.find_all('coordinates'): # Preliminary coordinates list needed
                coordinates_list.append(re.sub('<.*?>', '', str(coordinates)))

    for coord in coordinates_list:
        if coord.count(',') == 2: # If reading is a single coordinate (two commas detected)...
            coord = coord[:(len(coord)) - 2] # Delete separator characters ",0"
            long_lat = coord.split(',') # Split by comma
            longitude_list.append(long_lat[0]) # Add to latitude and longitude stationary lists
            latitude_list.append(long_lat[1])
            linestring_coord_list.append('') # Blank reading added to transitory list
        else:
            longitude_list.append('') # Adds blank readings to lat/long for transitory readings
            latitude_list.append('')
            linestring_coord_list.append(coord) # Reading added to transitory list

    columns_list = ['name','address','description','TimeSpan/begin','TimeSpan/end','longitude','latitude','LineString/coordinates']
    data_list = [name_list,address_list,description_list,time_begin_list,time_end_list,longitude_list,latitude_list,linestring_coord_list]
    dictionary = dict(zip(columns_list, data_list))
    formatted_location_df = pd.DataFrame(dictionary)
    formatted_location_df.to_csv('working_location.csv')

    # Exports trimmed dataframes as new CSV
    formatted_location_csv = formatted_location_df.to_csv('working_location.csv')
    
    return formatted_location_csv

In [ ]:
def Postgres_create_DB():
    try:
        # Creates connection to new DB, creates if nonexistant
        conn = psycopg2.connect(
            database=os.getenv('POSTGRES_DB'),
            user=os.getenv('POSTGRES_USER'),
            password=os.getenv('POSTGRES_PASSWORD'),
            host=os.getenv('POSTGRES_HOST')
        )

        # Cursor set for SQL insertion
        cursor = conn.cursor()
        print("Connected...")

        # DB tables to house both BPM 'Heart', Calories, and Geographic 'Trip' data are created
        # Primary integer key is set for all tables
        hearttable_sql='''
                        CREATE SEQUENCE heart_timeID_seq
                        START WITH 0
                        INCREMENT BY 1
                        MINVALUE 0
                        NO MAXVALUE
                        CACHE 1;
        
                        CREATE TABLE IF NOT EXISTS
                            heart(
                                time_id INTEGER PRIMARY KEY DEFAULT nextval('heart_timeID_seq'),
                                dateTime TIMESTAMP,
                                Bpm INTEGER,
                                Confidence INTEGER
                            );
                    '''
        cursor.execute(hearttable_sql)
        conn.commit()
        print("Created HEART Table...")

        calorietable_sql='''
                        CREATE SEQUENCE calorie_timeID_seq
                        START WITH 0
                        INCREMENT BY 1
                        MINVALUE 0
                        NO MAXVALUE
                        CACHE 1;
        
                        CREATE TABLE IF NOT EXISTS
                            calorie(
                                time_id INTEGER PRIMARY KEY DEFAULT nextval('calorie_timeID_seq'),
                                dateTime TIMESTAMP,
                                Calories FLOAT
                            );
                    '''
        cursor.execute(calorietable_sql)
        conn.commit()
        print("Created CALORIE Table...")

        triptable_sql='''
                        CREATE SEQUENCE trip_timeID_seq
                        START WITH 0
                        INCREMENT BY 1
                        MINVALUE 0
                        NO MAXVALUE
                        CACHE 1;
        
                        CREATE TABLE IF NOT EXISTS
                            trip(
                                time_id INTEGER PRIMARY KEY DEFAULT nextval('trip_timeID_seq'),
                                name TEXT,
                                address TEXT,
                                description TEXT,
                                TimeSpan_begin TIMESTAMP,
                                TimeSpan_end TIMESTAMP,
                                calories DOUBLE PRECISION,
                                avg_hr NUMERIC,
                                avg_conf NUMERIC,
                                longitude FLOAT,
                                latitude FLOAT,
                                LineString_coordinates TEXT
                            );
                    '''
        cursor.execute(triptable_sql)
        conn.commit()
        print("Created TRIP Table...")

    except psycopg2.Error as error:
            print("Modular Error...")
            print(error)

    # Database resources are closed at the end of insertion cycles
    finally:
        if cursor:
            cursor.close()
            if conn:
                conn.close()

In [ ]:
# Utility establishing database connection,cursor function returning all access objects
def connect_PostgreSQL():
    try:
        dbconn = psycopg2.connect(
            database=os.getenv('POSTGRES_DB'),
            user=os.getenv('POSTGRES_USER'),
            password=os.getenv('POSTGRES_PASSWORD'),
            host=os.getenv('POSTGRES_HOST')
        )
        cursor = dbconn.cursor()
        #print("Connected to 'heartcaltrip' database")
    except psycopg2.Error as error:
        print("Error opening database")
        dbconn = handle_DB_error(dbconn, cursor)
    return dbconn, cursor

# Utility to close database resources to call after future insertions
def close_DB_resources(dbconn, cursor):
    try:
        if dbconn:
            dbconn.close()
        if cursor:
            cursor.close()
        #print("closed resources")
    except psycopg2.Error as err:
        ("Error closing resources")

# Utility to handle database errors
def handle_DB_error(dbconn, cursor):
    if dbconn:
        try:
            dbconn.rollback()
            print("Rolled back transation")
        except psycopg2.Error as error:
            print("Error rolling back transaction")
        finally:
            close_DB_resources(dbconn, cursor)
            dbconn = None 
            return dbconn

In [ ]:
def Pg_insert_heart_csv(heart_csv):
    
    heart_insert = '''INSERT INTO heart (time_id, dateTime, Bpm, Confidence)
                      VALUES (DEFAULT,%s,%s,%s);
                    '''

    # BRM 'Heart' table is populated first from CSV...
    dbconn, cursor = connect_PostgreSQL()

    with open(heart_csv) as file:
        time_stamps = csv.DictReader(file)
        value_id = 0
        for time in time_stamps:
            value_id += 1
            record = (time['dateTime'], time['BPM'], time['Confidence'])
            cursor.execute(heart_insert, record)
            dbconn.commit()
        #print('All file records committed to database: BPM/HeartRate table...')

    close_DB_resources(dbconn, cursor)

In [ ]:
def Pg_insert_calorie_csv(calorie_csv):
    
    calorie_insert = '''INSERT INTO calorie (time_id, dateTime, Calories)
                        VALUES (DEFAULT,%s,%s);
                    '''
    # BRM 'Heart' table is populated first from CSV...
    dbconn, cursor = connect_PostgreSQL()

    with open(calorie_csv) as file:
        time_stamps = csv.DictReader(file)
        value_id = 0
        for time in time_stamps:
            value_id += 1
            record = (time['dateTime'], time['Calories'])
            cursor.execute(calorie_insert, record)
            dbconn.commit()
        #print('All file records committed to database: Caloric table...')

    close_DB_resources(dbconn, cursor)

In [ ]:
def Pg_insert_location_csv(Google_location_csv):
    
    trip_insert = '''INSERT INTO trip (time_id, name, address, description, TimeSpan_begin, TimeSpan_end, longitude, latitude, LineString_coordinates)
                     VALUES (DEFAULT,%s,%s,%s,%s,%s,%s,%s,%s);
                    '''
    
    dbconn, cursor = connect_PostgreSQL()

    with open(Google_location_csv) as file:
        time_stamps = csv.DictReader(file)
        value_id = 0
        for time in time_stamps:
            try:
                float(time['longitude'])
            except:
                #print('invalid entry')
                time['longitude'] = None
                time['latitude'] = None

            record = (time['name'], time['address'], time['description'], time['TimeSpan/begin'], time['TimeSpan/end'], time['longitude'], time['latitude'], time['LineString/coordinates'])
            cursor.execute(trip_insert, record)
            dbconn.commit()
        #print('All file records committed to database: Geographic/Location table...')

    close_DB_resources(dbconn, cursor)

In [ ]:
def Postgres_combine_view():
    # Readings within 'dateTime' are truncated down to their minute value
    # Average values for BPM and Confidence readings are taken per minute period
    # Summation of Calorie values are taken over for every minute period
    # All values are singularly grouped for every minute, then ordered chronologically...

    # These values are populated as single postgres View resource...
    create_calorie_heartrate_view = '''CREATE VIEW calorie_heartrate AS
                     SELECT date_trunc('minute', h.dateTime) AS dateTime_min,
                     avg(h.Bpm) AS BPM_min_avg, avg(h.Confidence) AS Conf_min_avg,
                     avg(c.calories) AS calories
                     FROM heart AS h
                     JOIN calorie as c
                     ON c.dateTime = date_trunc('minute', h.dateTime)
                     GROUP BY date_trunc('minute', h.dateTime)
                     ORDER BY date_trunc('minute', h.dateTime);
                  ''' 

    # All newly-generated minute values for Calories, BPM, Confidence are collected from View...
    pull_calorie_heartrate = '''SELECT dateTime_min, calories, BPM_min_avg, Conf_min_avg FROM calorie_heartrate;''' 

    # First insertion is executed, compended postgres View is created of most-relevant data
    # View is then fetched, that data is then stored in a single tuple...

    dbconn, cursor = connect_PostgreSQL()

    cursor.execute(create_calorie_heartrate_view)
    dbconn.commit()
    print('Grouping data readings per minute...')
    cursor.execute(pull_calorie_heartrate)
    tupples = cursor.fetchall()
    print('Extracting data approximations from Postgres...')

    close_DB_resources(dbconn, cursor)
    
    # That tuple is used to create new pd Dataframe of Calories + Average HeartRate per minute...
    calorie_heartrate_df = pd.DataFrame(tupples, columns=['dateTime','calories','bpm_min','conf_min'])
    return calorie_heartrate_df

In [ ]:
def Postgres_populate_Trip_table():
    
    dbconn, cursor = connect_PostgreSQL()

    trip_pull = '''SELECT * FROM trip
                   ORDER BY time_id;'''

    heart_cal_pull = '''SELECT sum(c_hr.calories) AS calories,
                               avg(c_hr.bpm_min_avg) AS avg_bpm,
                               avg(c_hr.conf_min_avg) AS confidence
                        FROM calorie_heartrate AS c_hr
                        WHERE c_hr.dateTime_min
                        BETWEEN (%s) AND (%s);
                     '''

    trip_fill_columns =  '''UPDATE public.trip
                            SET calories = CAST(%s AS DOUBLE PRECISION),
                                avg_hr = (%s),
                                avg_conf = (%s)
                            WHERE
                                time_id = (%s);
                         '''

    insertion_record = '''SELECT calories, avg_hr, avg_conf FROM trip
                          WHERE time_id = (%s);
                          '''

    final_trip_pull = '''SELECT time_id, name, address, timespan_begin, timespan_end,
                         calories, avg_hr, avg_conf, longitude, latitude, linestring_coordinates, description
                         FROM trip ORDER BY time_id;
                         '''

    # Location 'Trip' table is scanned, iterated by its consecutive time periods at a location (or in transit)

    # For each arbitrary time period expressed:
    #    - Our Postgres View of minutely-segregated calories burned, and average BPM readings, are collected.
    #        - Each per-minute reading is then again either summated or averaged to a single value.
    #        - These values are used for insertion into their respective column on 'Trip' table, via 'heart_cal_period'.
    #        - 'heart_cal_period' matches values within their appropriate time period.

    cursor.execute(trip_pull)
    trip_record = cursor.fetchall()

    for row in trip_record:
        cursor.execute(heart_cal_pull, (row[4], row[5]))
        heart_cal_period = cursor.fetchone()
        if heart_cal_period:
            cursor.execute(trip_fill_columns, (heart_cal_period[0], heart_cal_period[1], heart_cal_period[2], row[0]))
            cursor.execute(insertion_record, (row[0],))
            #print(cursor.fetchone())

    print("\nColumns populated: 'calories', 'average_hr', 'avg_conf'")
    print("Total rows are: ", len(trip_record))

    dbconn.commit()

    cursor.execute(final_trip_pull)
    trip_tupple = cursor.fetchall()

    close_DB_resources(dbconn, cursor)
    
    total_trip_df = pd.DataFrame(trip_tupple, columns=['datetime_min','bpm_min_avg','conf_min_avg','calories','Time end','Calories','BPM avg','BPM conf','Longitude','Latitude','LineString_coord','Description'])
    total_trip_df.to_csv('finished_table.csv')
    # pandas dataframe for calories and heartrate also generated, if needed...
    return total_trip_df

In [ ]:
def main():
    
    Postgres_create_DB()
    heart_jsons, calorie_jsons, location_kmls = generate_Fitbit_json_file_lists()
    
    file_id = 0
    
    for file in heart_jsons:
        heart_result_csv = json_heart_convert_csv(file)
        Pg_insert_heart_csv('working_heart.csv')
        file_id += 1
        print('Heart file', file_id, 'committed...')
    print('Heart rate data committed to database.\n')
    file_id = 0
    
    for file in calorie_jsons:
        calorie_result_csv = json_calorie_convert_csv(file)
        Pg_insert_calorie_csv('working_calorie.csv')
        file_id += 1
        print('Caloric file', file_id, 'committed...')
    print('Caloric data committed to database.\n')
    file_id = 0
    
    for file in location_kmls:
        formatted_location_csv = csv_format_google_kml(file)
        Pg_insert_location_csv('working_location.csv')
        file_id += 1
        print('Location file', file_id, 'committed...')
    print('Location data committed to database.\n')

    Postgres_combine_view()
    Postgres_populate_Trip_table()

In [ ]:
main()

In [ ]:
# Full CSV is loaded, duplicates are dropped by matching 'Time begin' values
# Rows are sorted by 'Time begin' value, extra hanging indices are dropped

total = (pd.read_csv('finished_table.csv')).drop_duplicates(['Time begin'])
total = (total.sort_values(['Time begin'])).reset_index()
total_trip_df = (total.drop(['index','Unnamed: 0','time_id'],1))

pd.set_option('display.max_rows', None)

# Empty Fitbit cells are identified, set to average of all values of that type

null_rows_cal = total_trip_df['Calories'].isnull()
null_rows_avg = total_trip_df['BPM avg'].isnull()
null_rows_conf = total_trip_df['BPM conf'].isnull()

for i, value in enumerate(total_trip_df['Calories']):
    if null_rows_cal[i]:
        total_trip_df['Calories'][i] = (total_trip_df['Calories']).mean()
for i, value in enumerate(total_trip_df['BPM avg']):
    if null_rows_avg[i]:
        total_trip_df['BPM avg'][i] = (total_trip_df['BPM avg']).mean()
for i, value in enumerate(total_trip_df['BPM conf']):
    if null_rows_conf[i]:
        total_trip_df['BPM conf'][i] = (total_trip_df['BPM conf']).mean()

In [ ]:
import folium
from folium import plugins
from folium.plugins import MarkerCluster,HeatMap
from folium import Marker
from branca.element import Figure
import math

In [ ]:
# Creates separate dataframes for stationary and transit periods
# Stationary/transit periods are separated by null value in "Lat/Long" columns
# Stationary Lat+Long are simply listed with BPM value (scaled for map)
# Transit period "LineString_coord" values are isolated, split by their dividers
# Transit Lat+Long are made into lists all sharing BPM for that time period

stationary_rows = []
stationary_readings = []
transit_rows = []
transit_readings = []

for idx, row in total_trip_df.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        st_row = [row['Name'], row['Address'], row['Time begin'], row['Time end'], row['Calories'], row['BPM avg'], row['BPM conf'], row['Longitude'], row['Latitude'], row['LineString_coord'], row['Description']]
        stationary_rows.append(st_row)
        stationary_trip_df = pd.DataFrame(stationary_rows, columns=['Name','Address','Time begin','Time end','Calories','BPM avg','BPM conf','Longitude','Latitude','LineString_coord','Description'])
        stationary_trip_df = stationary_trip_df.sort_values('Time begin')
        st_readings = [row['Latitude'], row['Longitude'], ((row['BPM avg'])/10000)]
        stationary_readings.append(st_readings)
    else:
        tr_row = [row['Name'], row['Address'], row['Time begin'], row['Time end'], row['Calories'], row['BPM avg'], row['BPM conf'], row['Longitude'], row['Latitude'], row['LineString_coord'], row['Description']]
        transit_rows.append(tr_row)
        transit_trip_df = pd.DataFrame(transit_rows, columns=['Name','Address','Time begin','Time end','Calories','BPM avg','BPM conf','Longitude','Latitude','LineString_coord','Description'])
        transit_trip_df = transit_trip_df.sort_values('Time begin')
        tran_reading = row['LineString_coord']
        tran_list = tran_reading.split()
        for t in tran_list:
            new = (t[:-2]).split(',')
            for t in new:
                lat_float = float(new[1])
                long_float = float(new[0])
            tran_reading = (lat_float, long_float, ((row['BPM avg'])/10000))
            transit_readings.append(tran_reading)
        for t in transit_readings:
            t = list(t)

In [ ]:
# Tuples are redefined as lists and fed (lat, long, bpm) into HeatMap function

print('Stationary readings:', len(stationary_readings))
print('Transitory readings:', len(transit_readings))

for t in stationary_readings:
    t = list(t)
for t in transit_readings:
    t = list(t)
    
m=folium.Map(width=825,height=825,location=[39.7392, -104.9903],zoom_start=4)
plugins.HeatMap(stationary_readings).add_to(m)
plugins.HeatMap(transit_readings).add_to(m)
m

In [ ]:
total_trip_df

In [ ]:
stationary_trip_df

In [ ]:
transit_trip_df

In [ ]:
from arcgis.gis import *
gis = GIS("https://www.arcgis.com", "steven.wortmann", "Paigek#624")
m = gis.map("Pittsburgh, PA",mode='2D')
m

In [ ]:
df = [stationary_trip_df['Latitude'], stationary_trip_df['Longitude']]
points = gis.content.import_data(df)
m.add_layer(points)

In [ ]:
items = gis.content.search(query="landsat usa", item_type="Feature Layer", max_items = 100)
for item in items:
    display(item)

In [ ]:
from arcgis.gis import *

gis = GIS()
m = gis.map('United States')

#locations = [stationary_trip_df['Latitude'], stationary_trip_df['Longitude']]
#df = pd.DataFrame.from_records(locations)